In [2]:
import boto3
region = boto3.session.Session().region_name
bucket = 'resnet-bucket'
!aws s3api create-bucket --bucket {bucket} --create-bucket-configuration LocationConstraint={region}


An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [7]:
!aws s3 cp --recursive container/local_test/test_dir/ s3://{bucket}

upload: container/local_test/test_dir/input/config/resourceConfig.json to s3://resnet-bucket/input/config/resourceConfig.json
upload: container/local_test/test_dir/input/config/hyperparameters.json to s3://resnet-bucket/input/config/hyperparameters.json
upload: container/local_test/test_dir/input/data/training/x_test.npy to s3://resnet-bucket/input/data/training/x_test.npy
upload: container/local_test/test_dir/input/data/training/y_test.npy to s3://resnet-bucket/input/data/training/y_test.npy
upload: container/local_test/test_dir/input/data/training/y_train.npy to s3://resnet-bucket/input/data/training/y_train.npy
upload: container/local_test/test_dir/input/data/training/y_validation.npy to s3://resnet-bucket/input/data/training/y_validation.npy
upload: container/local_test/test_dir/input/data/training/x_train.npy to s3://resnet-bucket/input/data/training/x_train.npy
upload: container/local_test/test_dir/input/data/training/x_validation.npy to s3://resnet-bucket/input/data/training/x_v

In [3]:
account = boto3.client('sts').get_caller_identity().get('Account')
image_name = "resnet"
repositoryUri = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account, region, image_name)
repositoryUri

'239622400621.dkr.ecr.us-west-2.amazonaws.com/resnet'

In [12]:
from datetime import datetime
now = datetime.now()
job_name = 'resnet-' + now.strftime("%Y-%m-%d-%H-%M")
job_name

'resnet-2021-05-06-10-52'

In [9]:
import sagemaker
from sagemaker import get_execution_role
sagemaker_session = sagemaker.Session()
try:
    role = get_execution_role()
except Exception as e:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMakerRole')['Role']['Arn']
sm = boto3.client('sagemaker')
role

Couldn't call 'get_role' to get Role ARN from role name ikmr0004 to get Role path.


'arn:aws:iam::239622400621:role/SageMakerRole'

In [10]:
training='s3://{}/input/data/training/'.format(bucket)
modelpath='s3://{}/models/'.format(bucket)

AlgorithmSpecification = {
    'TrainingImage': repositoryUri,
    'TrainingInputMode': 'File',
    }
InputDataConfig = [
    {
        'ChannelName': 'training',
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': training,
                'S3DataDistributionType': 'FullyReplicated',
            },
        },
        'InputMode': 'File'
    }
]
OutputDataConfig = {
    'S3OutputPath': modelpath
    }
ResourceConfig = {
    'InstanceType': 'ml.c5.4xlarge',
    'InstanceCount': 1,
    'VolumeSizeInGB': 30,
    }
StoppingCondition = {
    'MaxRuntimeInSeconds': 60*60*12,
    }

In [13]:
response = sm.create_training_job(
    TrainingJobName = job_name,
    AlgorithmSpecification = AlgorithmSpecification,
    RoleArn = role,
    InputDataConfig = InputDataConfig,
    OutputDataConfig = OutputDataConfig,
    ResourceConfig = ResourceConfig,
    StoppingCondition = StoppingCondition
)
response

{'TrainingJobArn': 'arn:aws:sagemaker:us-west-2:239622400621:training-job/resnet-2021-05-06-10-52',
 'ResponseMetadata': {'RequestId': 'd17d5308-d48d-42ae-9706-c8c975941c04',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd17d5308-d48d-42ae-9706-c8c975941c04',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Thu, 06 May 2021 01:52:41 GMT'},
  'RetryAttempts': 0}}

In [8]:
import time
sm = boto3.client('sagemaker')

result = 1
while result is not 0:
    result = 0    
    respones = sm.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print("job status:{}".format(respones))
    status = 0 if respones == 'Completed' else 1
    result = result + status
    time.sleep(20)
print('all job completed')

Error: Pip module Unable to parse debugpy output, please log an issue with https://github.com/microsoft/vscode-jupyter is required for debugging cells. You will need to install it to debug cells.